# TODO

* Cleanup notebook
* Add API calls to retrieve endpoint settings after each step
* Perhaps add screenshots?

# Setup
---

In [1]:
%cd /root/rollout-strategies/

/root/rollout-strategies


In [2]:
import pprint

import boto3
import pandas as pd
from sagemaker import get_execution_role, s3, estimator
import sagemaker

BUCKET = 'talk-20200720'
LOCAL_DATA_DIRECTORY = './data'
print(f"Artifacts will be written to s3://{BUCKET}")

# Session variables we'll use throughout the notebook
sm_session = sagemaker.Session()
boto_session = sm_session.boto_session
sagemaker_client = boto_session.client('sagemaker')
role = get_execution_role()
print(f'Role ARN: {role}')

Artifacts will be written to s3://talk-20200720
Role ARN: arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200618T144956


## Upload Training and Validation Data to S3
---

To see how the dataset was preprocessed, see this notebook: [XGBoost customer churn notebook that starts with the original dataset](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb). 

In [3]:
data = pd.read_csv(f'{LOCAL_DATA_DIRECTORY}/training-dataset-with-header.csv')

data.head()

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,106,0,274.4,120,198.6,82,160.8,62,6.0,...,0,0,0,0,0,1,1,0,1,0
1,0,28,0,187.8,94,248.6,86,208.8,124,10.6,...,0,0,1,0,1,0,1,0,1,0
2,1,148,0,279.3,104,201.6,87,280.8,99,7.9,...,0,0,0,0,1,0,1,0,1,0
3,0,132,0,191.9,107,206.9,127,272.0,88,12.6,...,0,0,0,0,0,1,1,0,1,0
4,0,92,29,155.4,110,188.5,104,254.9,118,8.0,...,0,0,0,0,0,1,1,0,0,1


In [3]:
DATA_PREFIX = 'data'

s3_input_train = sm_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/train.csv',
                                        bucket=BUCKET,
                                        key_prefix=DATA_PREFIX)

s3_input_validation = sm_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/validation.csv',
                                             bucket=BUCKET,
                                             key_prefix=DATA_PREFIX)

s3_input_train = sagemaker.s3_input(s3_data=s3_input_train, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=s3_input_validation, content_type='csv')

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


## Train Two XGBoost Models using different Hyperparameter Settings
---

In [4]:
from sagemaker.amazon.amazon_estimator import get_image_uri

MODEL_PREFIX = 'models'

xgboost_image_uri = get_image_uri(boto_session.region_name, 'xgboost', repo_version='0.90-2')
print(f'Training models using Docker image: {xgboost_image_uri}')

# Train Model A
model_A = estimator.Estimator(image_name=xgboost_image_uri,
                              role=role,
                              train_instance_count=1,
                              train_instance_type='ml.m4.xlarge',
                              output_path=f"s3://{BUCKET}/{MODEL_PREFIX}/model-A",
                              base_job_name="model-A",
                              sagemaker_session=sm_session)

model_A.set_hyperparameters(max_depth=5,
                            subsample=0.8,
                            num_round=600,
                            eta=0.2,
                            gamma=4,
                            min_child_weight=6,
                            silent=0,
                            objective='binary:logistic')

# Train Model B
model_B = estimator.Estimator(image_name=xgboost_image_uri,
                              role=role,
                              train_instance_count=1,
                              train_instance_type='ml.m4.xlarge',
                              output_path=f"s3://{BUCKET}/{MODEL_PREFIX}/model-B",
                              base_job_name="model-B",
                              sagemaker_session=sm_session)

model_B.set_hyperparameters(max_depth=3,
                            subsample=0.5,
                            num_round=600,
                            eta=0.2,
                            gamma=4,
                            min_child_weight=6,
                            silent=0,
                            objective='binary:logistic')

# Set wait=False so both A and B can train simultaneously
model_A.fit({'train': s3_input_train, 'validation': s3_input_validation}, wait=False)
model_B.fit({'train': s3_input_train, 'validation': s3_input_validation})

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').


Training models using Docker image: 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3
2020-07-05 13:09:26 Starting - Starting the training job...
2020-07-05 13:09:28 Starting - Launching requested ML instances......
2020-07-05 13:10:29 Starting - Preparing the instances for training...
2020-07-05 13:11:18 Downloading - Downloading input data...
2020-07-05 13:11:39 Training - Downloading the training image...
2020-07-05 13:12:27 Uploading - Uploading generated training model
2020-07-05 13:12:27 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined 

## Deploy Models

We call the `Model.deploy` method to create separate `Endpoint`s, `EndpointConfig`, and `Model` objects. We will create a single `Endpoint` so we can delete the `Endpoint` objects created here.

In [8]:
model_A.deploy(initial_instance_count=1,
               instance_type="ml.m4.xlarge",
               endpoint_name='endpoint-model-A',
               model_name='model-A',
               wait=False)

model_B.deploy(initial_instance_count=1,
               instance_type="ml.m4.xlarge",
               endpoint_name='endpoint-model-B',
               model_name='model-B')

---------------!

### Describe the Endpoint and EndpointConfig

In [11]:
model_A_endpoint = sagemaker_client.describe_endpoint(
    EndpointName='endpoint-model-A')
pprint.pprint(model_A_endpoint)

{'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 35, 31000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/endpoint-model-a',
 'EndpointConfigName': 'endpoint-model-A',
 'EndpointName': 'endpoint-model-A',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 13, 29, 45, 953000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 22, 36, 726000, tzinfo=tzlocal()),
                                             'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
                                             'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'}],
                         'DesiredInstanceCount': 1

In [13]:
model_A_endpoint_config = sagemaker_client.describe_endpoint_config(
    EndpointConfigName=model_A_endpoint['EndpointConfigName'])
pprint.pprint(model_A_endpoint_config)

{'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 34, 888000, tzinfo=tzlocal()),
 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint-config/endpoint-model-a',
 'EndpointConfigName': 'endpoint-model-A',
 'ProductionVariants': [{'InitialInstanceCount': 1,
                         'InitialVariantWeight': 1.0,
                         'InstanceType': 'ml.m4.xlarge',
                         'ModelName': 'model-A',
                         'VariantName': 'AllTraffic'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '325',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 05 Jul 2020 13:31:15 GMT',
                                      'x-amzn-requestid': 'd9e1f75c-b8a8-41cd-969e-da5974b99990'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd9e1f75c-b8a8-41cd-969e-da5974b99990',
                      'RetryAttempts': 0}}


In [18]:
sagemaker_client.describe_model(ModelName='model-A')

{'ModelName': 'model-A',
 'PrimaryContainer': {'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://talk-20200720/models/model-A/model-A-2020-07-05-13-09-20-788/output/model.tar.gz',
  'Environment': {}},
 'ExecutionRoleArn': 'arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200618T144956',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 14, 461000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-2:209970524256:model/model-a',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': 'db3d30fb-4112-4336-9b8b-0758653b0af3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'db3d30fb-4112-4336-9b8b-0758653b0af3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '515',
   'date': 'Sun, 05 Jul 2020 13:34:30 GMT'},
  'RetryAttempts': 0}}

In [15]:
model_B_endpoint = sagemaker_client.describe_endpoint(EndpointName='endpoint-model-B')
pprint.pprint(model_B_endpoint)

{'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 38, 215000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/endpoint-model-b',
 'EndpointConfigName': 'endpoint-model-B',
 'EndpointName': 'endpoint-model-B',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 13, 29, 51, 278000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 22, 40, 59000, tzinfo=tzlocal()),
                                             'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
                                             'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'}],
                         'DesiredInstanceCount': 1

In [17]:
pprint.pprint(sagemaker_client.describe_endpoint_config(
    EndpointConfigName=model_B_endpoint['EndpointConfigName']))

{'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 38, 62000, tzinfo=tzlocal()),
 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint-config/endpoint-model-b',
 'EndpointConfigName': 'endpoint-model-B',
 'ProductionVariants': [{'InitialInstanceCount': 1,
                         'InitialVariantWeight': 1.0,
                         'InstanceType': 'ml.m4.xlarge',
                         'ModelName': 'model-B',
                         'VariantName': 'AllTraffic'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '325',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 05 Jul 2020 13:33:50 GMT',
                                      'x-amzn-requestid': 'a101f4c5-a8f7-4c77-82cc-932da17ad128'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a101f4c5-a8f7-4c77-82cc-932da17ad128',
                      'RetryAttempts': 0}}


In [19]:
sagemaker_client.describe_model(ModelName='model-B')

{'ModelName': 'model-B',
 'PrimaryContainer': {'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://talk-20200720/models/model-B/model-B-2020-07-05-13-09-20-994/output/model.tar.gz',
  'Environment': {}},
 'ExecutionRoleArn': 'arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200618T144956',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 22, 35, 444000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-2:209970524256:model/model-b',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': 'e3d8d948-c417-4a63-9a42-9e708aca92e4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e3d8d948-c417-4a63-9a42-9e708aca92e4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '515',
   'date': 'Sun, 05 Jul 2020 13:34:38 GMT'},
  'RetryAttempts': 0}}

# Blue/Green Deployment

![blue green deployment](./figures/blue-green.png)

[Figure Source](https://d1.awsstatic.com/whitepapers/architecture/wellarchitected-Machine-Learning-Lens.pdf)

### Step 1: Create an Endpoint with a single Production Variant

In [20]:
churn_endpoint_name='churn-model'

sagemaker_client.create_endpoint(
    EndpointName=churn_endpoint_name,
    EndpointConfigName = model_A_endpoint['EndpointConfigName']
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '07f540d3-9f3c-43ce-8fbd-445abb5ccbb6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '07f540d3-9f3c-43ce-8fbd-445abb5ccbb6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 13:35:01 GMT'},
  'RetryAttempts': 0}}

In [24]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'endpoint-model-A',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 35, 3, 247000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 35, 1, 424000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 13, 42, 13, 999000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'b3a92b07-a60c-480c-b68a-669fa98cb18a',
  'HTTPStatusCode': 2

### Step 2. Create a new endpoint configuration, using the same production variants for the existing live model and for the new model.

In [22]:
variant_A = {'ModelName': 'model-A',
            'VariantName': 'blue',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium',
            'InitialVariantWeight': 1.0}

variant_B = {'ModelName': 'model-B',
            'VariantName': 'green',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium',
            'InitialVariantWeight': 0.0}

blue_green_endpoint_config = 'blue-green'
sagemaker_client.create_endpoint_config(EndpointConfigName = blue_green_endpoint_config,
                                        ProductionVariants = [variant_A, variant_B])

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint-config/blue-green',
 'ResponseMetadata': {'RequestId': '93de25de-661a-4fc2-a5f4-9d557db4b245',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '93de25de-661a-4fc2-a5f4-9d557db4b245',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Sun, 05 Jul 2020 13:39:01 GMT'},
  'RetryAttempts': 0}}

In [25]:
sagemaker_client.describe_endpoint_config(EndpointConfigName=blue_green_endpoint_config)

{'EndpointConfigName': 'blue-green',
 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint-config/blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'ModelName': 'model-A',
   'InitialInstanceCount': 1,
   'InstanceType': 'ml.t2.medium',
   'InitialVariantWeight': 1.0},
  {'VariantName': 'green',
   'ModelName': 'model-B',
   'InitialInstanceCount': 1,
   'InstanceType': 'ml.t2.medium',
   'InitialVariantWeight': 0.0}],
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 39, 2, 576000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '702b35e2-7852-46d7-9101-8b2e5a1988a2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '702b35e2-7852-46d7-9101-8b2e5a1988a2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '435',
   'date': 'Sun, 05 Jul 2020 13:42:31 GMT'},
  'RetryAttempts': 0}}

### Step 3. Update the existing live endpoint with the new endpoint configuration. Amazon SageMaker creates the required infrastructure for the new production variant and updates the weights without any downtime.

In [26]:
sagemaker_client.update_endpoint(EndpointName=churn_endpoint_name,
                                 EndpointConfigName=blue_green_endpoint_config)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '38f7f87c-9967-47ee-a180-3607668a08f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '38f7f87c-9967-47ee-a180-3607668a08f8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 13:42:44 GMT'},
  'RetryAttempts': 0}}

In [33]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 42, 46, 912000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1},
  {'VariantName': 'green',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     '

### Step 4. Switch traffic to the new model through an API call.

In [34]:
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=churn_endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'blue',
            'DesiredWeight': 0.0
        },
        {
            'VariantName': 'green',
            'DesiredWeight': 1.0
        }
    ]
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '2fe12bcb-52e7-473b-968d-a3b3fe12f52b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2fe12bcb-52e7-473b-968d-a3b3fe12f52b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 13:50:36 GMT'},
  'RetryAttempts': 0}}

In [36]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 42, 46, 912000, tzinfo=tzlocal())}],
   'CurrentWeight': 0.0,
   'DesiredWeight': 0.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1},
  {'VariantName': 'green',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     '

### Step 5. Create a new endpoint configuration with only the new production variant and apply it to the endpoint. 

In [37]:
sagemaker_client.update_endpoint(EndpointName=churn_endpoint_name,
                                 EndpointConfigName=model_B_endpoint['EndpointConfigName'])

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': 'e55f7438-3a5e-4cc4-b048-8f59876bde96',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e55f7438-3a5e-4cc4-b048-8f59876bde96',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 13:52:10 GMT'},
  'RetryAttempts': 0}}

In [55]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'endpoint-model-B',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 13, 52, 13, 156000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 35, 1, 424000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 13, 59, 19, 838000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'b372ed4c-44e8-4f53-aa39-12bfdd32248f',
  'HTTPStatusCode': 

# Canary Deployment

TODO Add picture

### Step 0. Reset Endpoint to use Model-A

In [56]:
sagemaker_client.update_endpoint(EndpointName=churn_endpoint_name,
                                 EndpointConfigName=model_A_endpoint['EndpointConfigName'])

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '57824213-990b-4f17-b060-83500686b20d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '57824213-990b-4f17-b060-83500686b20d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 17:20:29 GMT'},
  'RetryAttempts': 0}}

In [58]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'endpoint-model-A',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 17, 20, 32, 212000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 35, 1, 424000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 17, 27, 38, 510000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'fb5d8adc-f55a-4698-a9b3-75662ef5a1f2',
  'HTTPStatusCode': 

### Step 2. Update the existing live endpoint with the new endpoint configuration. Amazon SageMaker creates the required infrastructure for the new production variant and updates the weights without any downtime.

In [60]:
sagemaker_client.update_endpoint(EndpointName=churn_endpoint_name,
                                 EndpointConfigName=blue_green_endpoint_config)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': 'a2aef234-6d7b-4765-ab09-48e9ec56516b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a2aef234-6d7b-4765-ab09-48e9ec56516b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 17:29:19 GMT'},
  'RetryAttempts': 0}}

In [61]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 17, 29, 22, 171000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1},
  {'VariantName': 'green',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     '

### Step 3. Gradually move over traffic.

In [62]:
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=churn_endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'blue',
            'DesiredWeight': 0.75
        },
        {
            'VariantName': 'green',
            'DesiredWeight': 0.25
        }
    ]
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '3842cb36-043c-4695-9dae-e7c3d2add52b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3842cb36-043c-4695-9dae-e7c3d2add52b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 17:42:08 GMT'},
  'RetryAttempts': 0}}

In [63]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 17, 29, 22, 171000, tzinfo=tzlocal())}],
   'CurrentWeight': 0.75,
   'DesiredWeight': 0.75,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1},
  {'VariantName': 'green',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
    

In [64]:
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=churn_endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'blue',
            'DesiredWeight': 0.25
        },
        {
            'VariantName': 'green',
            'DesiredWeight': 0.75
        }
    ]
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': 'be2375d3-2938-402b-8466-76bef818f4d5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'be2375d3-2938-402b-8466-76bef818f4d5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 17:54:59 GMT'},
  'RetryAttempts': 0}}

In [65]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'blue-green',
 'ProductionVariants': [{'VariantName': 'blue',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 17, 29, 22, 171000, tzinfo=tzlocal())}],
   'CurrentWeight': 0.25,
   'DesiredWeight': 0.25,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1},
  {'VariantName': 'green',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
    

### Step 4. Create a new endpoint configuration with only the new production variant and apply it to the endpoint. 

In [66]:
sagemaker_client.update_endpoint(EndpointName=churn_endpoint_name,
                                 EndpointConfigName=model_B_endpoint['EndpointConfigName'])

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': 'edf8f841-e343-4917-9e74-b74f2db54668',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'edf8f841-e343-4917-9e74-b74f2db54668',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sun, 05 Jul 2020 18:31:01 GMT'},
  'RetryAttempts': 0}}

In [67]:
sagemaker_client.describe_endpoint(EndpointName=churn_endpoint_name)

{'EndpointName': 'churn-model',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'endpoint-model-B',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
     'ResolutionTime': datetime.datetime(2020, 7, 5, 18, 31, 4, 298000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 7, 5, 13, 35, 1, 424000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 7, 5, 18, 37, 11, 327000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5e3ef212-5eb0-446d-9fe1-60b6d23a5527',
  'HTTPStatusCode': 2